In [11]:
from pymongo import MongoClient
import pandas as pd
import re
import folium

In [12]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [13]:
db, coll = connectCollection('companies','companies')
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [14]:
interesting_companies = list(coll.find({"$and":
                                        [{"founded_year":{"$gte":2009}},
                                        {"deadpooled_year":None}]}))

print(len(interesting_companies))


792


In [23]:
interesting_companies = list(coll.find({"$and":
                                        [{"founded_year":{"$gte":2009}},
                                        {"deadpooled_year":None},
                                        {"total_money_raised":{"$exists": True,"$ne":"$0"}},
                                        {"offices.latitude": {"$exists": True, "$ne": None}},
                                        {"offices.longitude": {"$exists": True, "$ne": None}},
                                        {"$or":
                                        [{"category_code":"web"},
                                        {"category_code":"mobile"},
                                        {"category_code":"finance"},
                                        {"category_code":"games_video"},
                                        {"category_code":"analytics"},
                                        {"category_code": "software"}]}]},
                                      {"name":1,"founded_year": 1, "offices.latitude":1,
                                       "offices.longitude":1, "category_code":1, "location":1, 
                                       "offices":1,"total_money_raised":1}))
                                      
interesting_companies

[{'_id': ObjectId('52cdef7c4bab8bd675297f9e'),
  'name': 'GENWI',
  'category_code': 'mobile',
  'founded_year': 2010,
  'total_money_raised': '$7.1M',
  'offices': [{'description': '',
    'address1': '4966 El Camino Real',
    'address2': 'Suite 200',
    'zip_code': '94022',
    'city': 'Los Altos',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 33.8171,
    'longitude': -111.9035}]},
 {'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'category_code': 'web',
  'founded_year': 2013,
  'total_money_raised': '$8M',
  'offices': [{'description': '',
    'address1': 'One Franklin Parkway',
    'address2': 'Building 910, Suite 110',
    'zip_code': '94401',
    'city': 'San Mateo',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.566879,
    'longitude': -122.323895}]},
 {'_id': ObjectId('52cdef7c4bab8bd67529831a'),
  'name': 'Social Gaming Network',
  'category_code': 'games_video',
  'founded_year': 2011,
  'total_money_raised': 

In [26]:
interesting_companies[2]

{'_id': ObjectId('52cdef7c4bab8bd67529831a'),
 'name': 'Social Gaming Network',
 'category_code': 'games_video',
 'founded_year': 2011,
 'total_money_raised': '$17.1M',
 'offices': [{'description': 'HQ Los Angeles',
   'address1': '9570 W. Pico Blvd 3rd Floor',
   'address2': '',
   'zip_code': '90035',
   'city': 'Los Angeles',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.446823,
   'longitude': -122.161523},
  {'description': '',
   'address1': '9606 S. Santa Monica Blvd',
   'address2': '2nd Floor',
   'zip_code': '90210',
   'city': 'Beverly Hills',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 34.0698494,
   'longitude': -118.4054182}]}

In [27]:
def getLocation(companies):
    location =[]
    for i in range(len(companies['offices'])):
        longitude = companies['offices'][i]['longitude']
        latitude = companies['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location.append(loc)
    return location

getLocation(interesting_companies[2])

[{'type': 'Point', 'coordinates': [-122.161523, 37.446823]},
 {'type': 'Point', 'coordinates': [-118.4054182, 34.0698494]}]

In [28]:
for comp in interesting_companies:
    value = {"$set": {'location': getLocation(comp)}
    coll.update_many(comp, value)
    
interesting_companies[2]['location']
    
             



SyntaxError: invalid syntax (<ipython-input-28-f42c2e2ab80d>, line 3)

In [9]:
for c in company:
    for i in range(len(c['offices'])):
        value = {"$set": {'location'+str(i):getLocation(c,i)}}
        coll.update_one(c,value)

NameError: name 'company' is not defined

In [126]:
print(len(interesting_companies[2]['offices']))

2


In [87]:
'''def getLocation(company):
    coordinates = []
    for i in range(len(company["offices"])):
        longitude = company['offices'][i]['longitude']
        latitude = company['offices'][i]['latitude']
        coordinate = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }    
        coordinates.append(coordinate)
    return coordinates

getLocation(interesting_companies)
'''

'def getLocation(company):\n    coordinates = []\n    for i in range(len(company["offices"])):\n        longitude = company[\'offices\'][i][\'longitude\']\n        latitude = company[\'offices\'][i][\'latitude\']\n        coordinate = {\n            \'type\':\'Point\',\n            \'coordinates\':[longitude, latitude]\n        }    \n        coordinates.append(coordinate)\n    return coordinates\n\ngetLocation(interesting_companies)\n'

In [88]:

'''coordinates = [{'type': 'Point', 'coordinates':getLocation(comp)}for comp in interesting_companies]

print(coordinates)'''


"coordinates = [{'type': 'Point', 'coordinates':getLocation(comp)}for comp in interesting_companies]\n\nprint(coordinates)"

In [89]:
'''def getLocation(companies):
    location= []
    for i in range(len(interesting_companies["offices"])):
        latitude = interesting_companies["offices"][i]["latitude"]
        longitude = interesting_companies["offices"][i]["longitude"]
        loc = {
            'type':'Point',
            'coordinates':[float(longitude), float(latitude)]
        }    
        location.append(loc)
    return location


getLocation(interesting_companies[0])

for comp in interesting_companies:
    value = {"$set": {"location":getLocation(comp)}}
    coll.update_one(comp, value)'''

'def getLocation(companies):\n    location= []\n    for i in range(len(interesting_companies["offices"])):\n        latitude = interesting_companies["offices"][i]["latitude"]\n        longitude = interesting_companies["offices"][i]["longitude"]\n        loc = {\n            \'type\':\'Point\',\n            \'coordinates\':[float(longitude), float(latitude)]\n        }    \n        location.append(loc)\n    return location\n\n\ngetLocation(interesting_companies[0])\n\nfor comp in interesting_companies:\n    value = {"$set": {"location":getLocation(comp)}}\n    coll.update_one(comp, value)'

In [170]:
companies = pd.DataFrame(interesting_companies)
companies.head()


,_id,name,category_code,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297f9e,GENWI,mobile,2010,$7.1M,"[{'latitude': 33.8171, 'longitude': -111.9035}]"
1,52cdef7c4bab8bd675297fec,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323..."
2,52cdef7c4bab8bd67529831a,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161..."
3,52cdef7c4bab8bd675298509,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32..."
4,52cdef7c4bab8bd675298591,Handipoints,games_video,2009,$800k,"[{'latitude': 37.772323, 'longitude': -122.214..."


In [171]:
companies["coordinates"] = coordinates
companies.head()

,_id,name,category_code,founded_year,total_money_raised,offices,coordinates
0,52cdef7c4bab8bd675297f9e,GENWI,mobile,2010,$7.1M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,52cdef7c4bab8bd675297fec,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,52cdef7c4bab8bd67529831a,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,52cdef7c4bab8bd675298509,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
4,52cdef7c4bab8bd675298591,Handipoints,games_video,2009,$800k,"[{'latitude': 37.772323, 'longitude': -122.214...","{'type': 'Point', 'coordinates': [-122.214897,..."


In [175]:
companies["coordinates"][2]

{'type': 'Point', 'coordinates': [-122.161523, 37.446823]}

In [163]:
companies = companies.drop(['_id'], axis = 1)
companies.head()


,name,category_code,founded_year,total_money_raised,offices,coordinates
0,GENWI,mobile,2010,$7.1M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
4,Handipoints,games_video,2009,$800k,"[{'latitude': 37.772323, 'longitude': -122.214...","{'type': 'Point', 'coordinates': [-122.214897,..."


In [164]:
companies.total_money_raised = companies.total_money_raised.str.replace(r'(\d+)([A-Za-z])', r'\1 \2')
companies.total_money_raised = companies.total_money_raised.str.replace(r'(.)(\d+)', r'\1 \2')
companies.head()


,name,category_code,founded_year,total_money_raised,offices,coordinates
0,GENWI,mobile,2010,$ 7. 1 M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,Fixya,web,2013,$ 8 M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,Social Gaming Network,games_video,2011,$ 17. 1 M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,Skydeck,mobile,2012,$ 4 M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
4,Handipoints,games_video,2009,$ 800 k,"[{'latitude': 37.772323, 'longitude': -122.214...","{'type': 'Point', 'coordinates': [-122.214897,..."


In [165]:
#companies3 = companies2[companies2["total_money_raised"].str.contains("M", na=True)]
#companies3
#df[df['model'].str.contains('ac')]

companies1 = companies[['M' in x for x in companies['total_money_raised']]]
companies1.head()

,name,category_code,founded_year,total_money_raised,offices,coordinates
0,GENWI,mobile,2010,$ 7. 1 M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,Fixya,web,2013,$ 8 M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,Social Gaming Network,games_video,2011,$ 17. 1 M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,Skydeck,mobile,2012,$ 4 M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
5,Pixsta,web,2010,£ 3 M,"[{'latitude': 51.51819, 'longitude': -0.208979}]","{'type': 'Point', 'coordinates': [-0.208979, 5..."


In [169]:
print(companies1["coordinates"])


0     {'type': 'Point', 'coordinates': [-111.9035, 3...
1     {'type': 'Point', 'coordinates': [-122.323895,...
2     {'type': 'Point', 'coordinates': [-122.161523,...
3     {'type': 'Point', 'coordinates': [-122.3254705...
5     {'type': 'Point', 'coordinates': [-0.208979, 5...
6     {'type': 'Point', 'coordinates': [-71.24762, 4...
7     {'type': 'Point', 'coordinates': [-113.515769,...
13    {'type': 'Point', 'coordinates': [-86.0547016,...
14    {'type': 'Point', 'coordinates': [-73.987764, ...
16    {'type': 'Point', 'coordinates': [-122.0385792...
17    {'type': 'Point', 'coordinates': [-121.955879,...
18    {'type': 'Point', 'coordinates': [-122.4041485...
19    {'type': 'Point', 'coordinates': [-72.5196907,...
20    {'type': 'Point', 'coordinates': [-122.0780632...
21    {'type': 'Point', 'coordinates': [-71.24756, 4...
22    {'type': 'Point', 'coordinates': [-71.1469003,...
23    {'type': 'Point', 'coordinates': [-74.0136605,...
24    {'type': 'Point', 'coordinates': [-122.339

In [57]:
companies1.to_json(orient= "records")

'[{"name":"GENWI","category_code":"mobile","founded_year":2010,"total_money_raised":"$  7.  1 M","offices":[{"latitude":33.8171,"longitude":-111.9035}],"coordinates":{"type":"Point","coordinates":[-111.9035,33.8171]}},{"name":"Fixya","category_code":"web","founded_year":2013,"total_money_raised":"$  8 M","offices":[{"latitude":37.566879,"longitude":-122.323895}],"coordinates":{"type":"Point","coordinates":[-122.323895,37.566879]}},{"name":"Social Gaming Network","category_code":"games_video","founded_year":2011,"total_money_raised":"$  17.  1 M","offices":[{"latitude":37.446823,"longitude":-122.161523},{"latitude":34.0698494,"longitude":-118.4054182}],"coordinates":{"type":"Point","coordinates":[-122.161523,37.446823]}},{"name":"Skydeck","category_code":"mobile","founded_year":2012,"total_money_raised":"$  4 M","offices":[{"latitude":37.5645375,"longitude":-122.3254705}],"coordinates":{"type":"Point","coordinates":[-122.3254705,37.5645375]}},{"name":"Pixsta","category_code":"web","foun